In [2]:
import sys; sys.path.append('../../')
from DataPreparation.DataPreparation import read_data
from sklearn import svm
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from ModelPipelines.ModelAnalysis import recursive_feature_elimination
from utils import load_hyperparameters, save_model, get_metrics
from mlpath import mlquest as mlq
mlq.start_quest("SVM", table_dest="../../", log_defs=True)
model_name = 'SVM'


c:\Users\dell\AppData\Local\Programs\Python\Python38\lib\site-packages\mlpath\mlquest\mlquest.py:65: UserWarning: Attempting to start a run while another one is active may cause data overwrite
  if mlquest.active == True: warnings.warn("Attempting to start a run while another one is active may cause data overwrite")


### Read the data

In [3]:
x_data_d, y_data_d = mlq.l(read_data)(kind="Numerical")
opt_params = load_hyperparameters(model_name)

### Initiate SVM

In [4]:
clf = mlq.l(svm.SVC)(kernel='linear', C=53)

### Train & Evaluate SVM

In [5]:
cross_validation(mlq, clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1)

Leave-One-Out gives Accuracy: 0.9771186440677966
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       152
           1       0.95      0.92      0.94       160
           2       0.97      0.98      0.97       324
           3       0.99      0.99      0.99       544

    accuracy                           0.98      1180
   macro avg       0.97      0.97      0.97      1180
weighted avg       0.98      0.98      0.98      1180

10-Repeated 10-fold gives Accuracy: 0.9838983050847457
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       152
           1       0.96      0.94      0.95       160
           2       0.98      0.98      0.98       324
           3       1.00      1.00      1.00       544

    accuracy                           0.98      1180
   macro avg       0.98      0.98      0.98      1180
weighted avg       0.98      0.98      0.98      1180



### Evaluate Model Generalization

In [7]:
y_pred = mlq.l(cross_val_predict)(clf, x_data_d, y_data_d, cv=4)
val_metrics = classification_report(y_data_d, y_pred, digits=3)
val_acc, val_wf1 = get_metrics(val_metrics)
print(val_metrics)

              precision    recall  f1-score   support

           0      0.950     0.993     0.971       152
           1      0.960     0.900     0.929       160
           2      0.963     0.975     0.969       324
           3      0.994     0.993     0.994       544

    accuracy                          0.975      1180
   macro avg      0.967     0.965     0.966      1180
weighted avg      0.976     0.975     0.975      1180



c:\Users\dell\AppData\Local\Programs\Python\Python38\lib\site-packages\mlpath\mlquest\mlquest.py:115: UserWarning: Attempting to log a function when no run is active will do nothing
  warnings.warn("Attempting to log a function when no run is active will do nothing")


### Tracking

In [6]:
mlq.show_logs(quest_name="SVM", table_dest="../../", last_k=10)

AttributeError: type object 'mlquest' has no attribute 'show_logs'